In [2]:
% matplotlib inline
from pymongo import MongoClient
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import seaborn as sns
import networkx as nx
import nltk
from langdetect import detect
from langdetect import detect_langs
from langdetect import DetectorFactory
from nltk.tokenize import RegexpTokenizer  
from stop_words import get_stop_words  
from nltk.stem.porter import PorterStemmer  
from gensim import corpora, models  
import gensim  


C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [69]:
mongo_username = 'Campione'
mongo_password = 'veTRxJL29lpKWwPn'
mongo_url = 'mongodb://%s:%s@cluster0-shard-00-01-i6gcp.mongodb.net:27017/admin' % (
            mongo_username, mongo_password)
client = MongoClient(mongo_url,ssl=True,replicaSet='Cluster0-shard-0',authSource='admin')
info = client.server_info()  # Forces a call.
# print(info)
print(client.database_names())

['gitdb', 'gitdbPro', 'guidebook', 'test', 'admin', 'local']


In [4]:
db = client.gitdbPro
repos = db.repos
descriptions = repos.distinct('description')
df = pd.DataFrame(descriptions)
df.head()

,0
0,How to be low-level programmer
1,Import OpenStreetMap data into Unreal Engine 4
2,react-native template to target multiple platf...
3,Connect your App to Multiple Messaging Channel...
4,Proto Actor - Ultra fast distributed actors fo...


In [5]:
df[0]

0                           How to be low-level programmer
1           Import OpenStreetMap data into Unreal Engine 4
2        react-native template to target multiple platf...
3        Connect your App to Multiple Messaging Channel...
4        Proto Actor - Ultra fast distributed actors fo...
5                用rxjava简单封装了指纹识别，并实现对生命周期的自动绑定，顺便撸了一个指纹控件
6                                   pagehelper-spring-boot
7            📅 A date picker components for ionic2 ／ionic3
8        Track prices on Amazon and receive email alert...
9        Advanced web based trading client for the Stel...
10                                  Join our slack channel
11       An Elm-based prototype to help designers build...
12       Can neural networks transliterate Romaji into ...
13                              A repo with small projects
14               Go - CQRS / Event Sourcing made easy - Go
15                                RNN Tutorial for Artists
16       A macOS app to convert JSON objects into Swift.

In [6]:
DetectorFactory.seed = 0

In [7]:
cn_text_list = []
en_text_list = []
for item in df[0]:
    try:
        if detect(str(item)) == 'zh-cn':
            cn_text_list.append(item)
        else:
            en_text_list.append(item)
    except Exception as e:
        print(str(e))

No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.


In [8]:
len(cn_text_list)

697

In [9]:
len(en_text_list)

11143

In [70]:
df_cn = pd.DataFrame({'text':cn_text_list})
print(df_cn.head())
df_cn.to_csv('cn_text')

                                                text
0          用rxjava简单封装了指纹识别，并实现对生命周期的自动绑定，顺便撸了一个指纹控件
1           每周一篇，内容精简，不咸不淡，期盼探讨。微信公众号：芋道源码【纯源码分享公众号】
2                                  iOS即时通讯，从入门到“放弃”？
3  『假装看天气』─── 天气预报 & 公交查询 & 资讯阅读 & 妹纸福利 的 Android...
4                                         微信小程序－日历 📅


In [11]:
df_en = pd.DataFrame({'text':en_text_list})
print(df_en.head())
df_en.to_csv('en_text')

                                                text
0                     How to be low-level programmer
1     Import OpenStreetMap data into Unreal Engine 4
2  react-native template to target multiple platf...
3  Connect your App to Multiple Messaging Channel...
4  Proto Actor - Ultra fast distributed actors fo...


In [89]:
# GENERATE LDA Model

# list for tokenized documents in loop  
texts = []
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')
# Create p_stemmer of class PorterStemmer  
p_stemmer = PorterStemmer()

# loop through document list  
for i in en_text_list:
    if i is not None:
        raw = i.lower()
        # clean and tokenize document string
        tokens = tokenizer.tokenize(raw)
        
        # remove stop words from tokens  
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        
        # add tokens to list
        texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
      
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary, passes=20)

In [90]:
for topic in ldamodel.print_topics(num_words=5, num_topics=20):
    print(topic)

(0, '0.019*"framework" + 0.017*"book" + 0.016*"inform" + 0.015*"bot" + 0.013*"util"')
(1, '0.112*"android" + 0.074*"io" + 0.073*"app" + 0.030*"librari" + 0.016*"support"')
(2, '0.045*"list" + 0.031*"platform" + 0.028*"awesom" + 0.025*"resourc" + 0.024*"javascript"')
(3, '0.027*"architectur" + 0.021*"languag" + 0.020*"use" + 0.019*"search" + 0.016*"servic"')
(4, '0.023*"packag" + 0.021*"exploit" + 0.020*"type" + 0.017*"php" + 0.015*"cli"')
(5, '0.063*"data" + 0.024*"visual" + 0.017*"databas" + 0.016*"offici" + 0.015*"live"')
(6, '0.038*"program" + 0.021*"http" + 0.018*"org" + 0.018*"webpack" + 0.015*"0"')
(7, '0.044*"time" + 0.027*"detect" + 0.026*"web" + 0.021*"real" + 0.016*"object"')
(8, '0.069*"http" + 0.041*"com" + 0.041*"github" + 0.029*"file" + 0.013*"code"')
(9, '0.025*"css" + 0.025*"anim" + 0.022*"java" + 0.019*"test" + 0.016*"librari"')
(10, '0.050*"implement" + 0.049*"learn" + 0.034*"python" + 0.033*"deep" + 0.029*"model"')
(11, '0.027*"exampl" + 0.024*"collect" + 0.019*"api"

In [91]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
12     7.423709        1       1  0.218750  0.014579
10     6.716691        1       2 -0.132239  0.229948
13     6.381690        1       3 -0.126166 -0.026260
15     6.338094        1       4 -0.097567 -0.097746
11     5.953564        1       5 -0.115711 -0.063802
2      5.763358        1       6 -0.067453 -0.129455
16     5.517676        1       7 -0.070473  0.096964
17     5.139403        1       8 -0.054085 -0.111222
1      5.086509        1       9  0.022640 -0.063233
9      4.923859        1      10 -0.098780 -0.037496
19     4.798937        1      11 -0.048024 -0.075089
18     4.743170        1      12 -0.045387 -0.000273
14     4.413191        1      13 -0.037752  0.239460
4      4.073596        1      14  0.044258 -0.004760
8      3.989709        1      15  0.176345  0.040910
3      3.936421        1      16  0.040547  0.088919
7      3.890388        1      17  0.054307 -0.055363
6      3.790743        1      18  0.111096  0.001622
5      3.729437        1      19  0.108113 -0.005053
0      3.389855        1      20  0.117580 -0.042651, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
72    Default  480.000000     android  480.000000  30.0000  30.0000
179   Default  508.000000       react  508.000000  29.0000  29.0000
87    Default  421.000000          io  421.000000  28.0000  28.0000
249   Default  475.000000         app  475.000000  27.0000  27.0000
73    Default  317.000000          js  317.000000  26.0000  26.0000
31    Default  287.000000     network  287.000000  25.0000  25.0000
239   Default  296.000000        http  296.000000  24.0000  24.0000
78    Default  400.000000       learn  400.000000  23.0000  23.0000
37    Default  429.000000   implement  429.000000  22.0000  22.0000
340   Default  240.000000        data  240.000000  21.0000  21.0000
253   Default  220.000000        list  220.000000  20.0000  20.0000
276   Default  312.000000      compon  312.000000  19.0000  19.0000
292   Default  261.000000         vue  261.000000  18.0000  18.0000
90    Default  240.000000       swift  240.000000  17.0000  17.0000
350   Default  197.000000       manag  197.000000  16.0000  16.0000
288   Default  265.000000       sourc  265.000000  15.0000  15.0000
177   Default  225.000000       nativ  225.000000  14.0000  14.0000
384   Default  289.000000      python  289.000000  13.0000  13.0000
337   Default  227.000000          go  227.000000  12.0000  12.0000
1167  Default  173.000000        easi  173.000000  11.0000  11.0000
287   Default  216.000000        open  216.000000  10.0000  10.0000
950   Default  188.000000        deep  188.000000   9.0000   9.0000
91    Default  158.000000     written  158.000000   8.0000   8.0000
487   Default  145.000000        time  145.000000   7.0000   7.0000
238   Default  146.000000      github  146.000000   6.0000   6.0000
342   Default  165.000000       model  165.000000   5.0000   5.0000
237   Default  141.000000         com  141.000000   4.0000   4.0000
150   Default  583.000000     librari  583.000000   3.0000   3.0000
19    Default  148.000000    platform  148.000000   2.0000   2.0000
580   Default  161.000000          ui  161.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
3958  Topic20   15.754931       watch   16.559274   3.3346  -5.1973
1743  Topic20   15.208613         100   16.012956   3.3328  -5.2326
2071  Topic20   14.547513      propos   15.351857   3.3306  -5.2770
278   Topic20   14.262748       micro   15.067091   3.3295  -5.2968
3454  Topic20   13.827763         rpc   14.632106   3.3278  -5.3278
1735  Topic20   13.764942     symfoni   14.569285   3.3276  -5.3323
513   Topic20   13.675599          ml   14.479942   3.3272  -5.3388
996   Topic20   13.001455       tweak   13.805799   3.3244  -5.3894
332

In [52]:
# SKLEARN LEARN LDA
from __future__ import print_function
from time import time
from random import shuffle

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_samples = 7000
n_features = 100
n_components = 5
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [53]:
# shuffle the data
en_shuffle = en_text_list
shuffle(en_shuffle)
print(en_shuffle)
len(en_shuffle)

['An abnormal horizontal ListView-like pile layout with stretch and contraction effects.', 'Custom Visual Code Icons', '📝Next generation markdown editor, running on platforms of MacOS Windows and Linux.', 'This research aims at simply deploying CNN(Convolutional Neural Network) on mobile devices, with low complexity and high speed. ', 'This is the Pytorch implementation of Universal Style Transfer via Feature Transforms.', 'Stop searching for sample hashes on 10 different sites.', 'A bullet-proof, fast, and reliable headless browser API', 'RQAlpha 对接 vnpy 的扩展 Mod。通过启用该 Mod 来实现期货策略的实盘交易', '新的导航栏按钮位置偏移的解决方案,兼容iOS11,可自定义间距,同时兼容性更好', 'Own a piece of blockchain history!', 'JS and android ', 'bash shell script.', 'This is the code for "Learn Machine Learning in 3 Months" by Siraj Raval on Youtube ', 'Animated Social share buttons control for iOS written in Swift', 'Branchless UTF-8 decoder', 'TIC-80 is a fantasy computer for making, playing and sharing tiny games.', 'Linux Shell Scripts', 'Y

11143

In [54]:
# Get data sample
en_samples = en_shuffle[:n_samples]
len(en_samples)

7000

In [61]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(en_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
dtm_tf = tf_vectorizer.fit_transform(en_samples)
# tfid_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(en_samples)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for NMF...
done in 0.115s.
Extracting tf features for LDA...
done in 0.209s.


In [66]:
print(dtm_tf.shape)
print(dtm_tf)
print(dtm_tfidf.shape)
print(dtm_tfidf)

(7000, 100)
  (0, 51)	1
  (1, 13)	1
  (1, 22)	1
  (2, 53)	1
  (2, 98)	1
  (3, 35)	1
  (3, 61)	1
  (3, 63)	1
  (4, 82)	1
  (4, 40)	1
  (4, 75)	1
  (6, 2)	1
  (6, 9)	1
  (6, 30)	1
  (10, 1)	1
  (10, 44)	1
  (12, 48)	1
  (12, 55)	1
  (12, 13)	1
  (13, 84)	1
  (13, 99)	1
  (13, 41)	1
  (16, 53)	1
  (18, 8)	1
  (18, 80)	1
  :	:
  (6985, 41)	1
  (6986, 19)	1
  (6987, 72)	1
  (6987, 40)	1
  (6988, 19)	1
  (6988, 5)	1
  (6988, 60)	1
  (6988, 8)	1
  (6989, 70)	1
  (6991, 59)	1
  (6991, 76)	1
  (6992, 14)	1
  (6993, 20)	1
  (6993, 14)	1
  (6993, 18)	1
  (6993, 76)	1
  (6994, 94)	1
  (6994, 58)	1
  (6994, 37)	1
  (6994, 1)	2
  (6997, 60)	1
  (6998, 79)	1
  (6999, 97)	1
  (6999, 32)	1
  (6999, 78)	1
(7000, 100)
  (0, 51)	1.0
  (1, 22)	0.800646952851
  (1, 13)	0.599136425942
  (2, 98)	0.709758595298
  (2, 53)	0.704444984652
  (3, 63)	0.576421193456
  (3, 61)	0.561586131058
  (3, 35)	0.593598875621
  (4, 75)	0.56257212252
  (4, 40)	0.508329962023
  (4, 82)	0.652007098637
  (6, 30)	0.581554793603
  (

In [88]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(dtm_tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(lda, tfidf_feature_names, n_top_words)

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=7000 and n_features=100...
done in 0.217s.

Topics in NMF model (Frobenius norm):
Topic #0: android app ios view use project kotlin easy client design custom apps plugin java framework application github way native using
Topic #1: code source tool open project com 2017 line networks model https javascript swift framework github projects command package plugin google
Topic #2: library python javascript data lightweight fast easy like platform swift ui source create performance line css simple open api text
Topic #3: react simple js based vue native app component framework web components using api tool ios use node data built application
Topic #4: learning deep implementation python machine pytorch using tensorflow networks neural model network resources based image 2017 data project list text

Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=7000 and n_features=100...
do

In [68]:
# print(lda.components_)
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.sklearn.prepare(lda, dtm_tfidf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      21.801535        1       1  0.240366  0.292891
2      21.670345        1       2 -0.029008 -0.419460
1      20.856516        1       3 -0.391273 -0.097988
0      19.246924        1       4 -0.210219  0.370450
4      16.424680        1       5  0.390133 -0.145892, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
1     Default  201.000000         android  201.000000  30.0000  30.0000
49    Default  221.000000         library  221.000000  29.0000  29.0000
84    Default  163.000000           swift  163.000000  28.0000  28.0000
76    Default  178.000000           react  178.000000  27.0000  27.0000
88    Default  154.000000            tool  154.000000  26.0000  26.0000
41    Default  178.000000             ios  178.000000  25.0000  25.0000
48    Default  164.000000        learning  164.000000  24.0000  24.0000
80    Default  173.000000          simple  173.000000  23.0000  23.0000
74    Default  144.000000          python  144.000000  22.0000  22.0000
13    Default  169.000000            code  169.000000  21.0000  21.0000
44    Default  149.000000              js  149.000000  20.0000  20.0000
93    Default  150.000000           using  150.000000  19.0000  19.0000
95    Default  127.000000             vue  127.000000  18.0000  18.0000
32    Default  132.000000       framework  132.000000  17.0000  17.0000
3     Default  132.000000             app  132.000000  16.0000  16.0000
2     Default  116.000000             api  116.000000  15.0000  15.0000
40    Default  121.000000  implementation  121.000000  14.0000  14.0000
8     Default  172.000000           based  172.000000  13.0000  13.0000
97    Default  119.000000             web  119.000000  12.0000  12.0000
23    Default  115.000000            data  115.000000  11.0000  11.0000
24    Default   99.000000            deep   99.000000  10.0000  10.0000
81    Default   92.000000          source   92.000000   9.0000   9.0000
91    Default   89.000000             use   89.000000   8.0000   8.0000
54    Default   82.000000            list   82.000000   7.0000   7.0000
90    Default   69.000000              ui   69.000000   6.0000   6.0000
72    Default   83.000000         project   83.000000   5.0000   5.0000
59    Default   80.000000          native   80.000000   4.0000   4.0000
39    Default   82.000000           image   82.000000   3.0000   3.0000
17    Default   75.000000       component   75.000000   2.0000   2.0000
83    Default   68.000000         support   68.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
42     Topic5   46.565590            java   47.021433   1.7966  -3.2814
57     Topic5   45.573881      management   46.053968   1.7959  -3.3029
71     Topic5   43.754335     programming   44.216335   1.7959  -3.3436
45     Topic5   42.179632          kotlin   42.634559   1.7957  -3.3803
66     Topic5   41.477833         package   41.936739   1.7954  -3.3971
36     Topic5   40.108695            html   40.565110   1.7951  -3.4306
27     Topic5   37.928857          easily   38.394253   1.7942  -3.4865
6      Topic5   34.794028            apps   62.851361   1.2151  -3.5728
28     Topic5   44.416145            easy   82.003017   1.1932  -3.3286
49     Topic5    7.204887         library  221.361996  -1.6187  -5.1475
41     Topic5    0.746841             ios  178.329280  -3.6691  -7.4141
56     Topic5    0.137777            make   33.023397  -3.6729  -9.1044
78     Topic5    0.138291        security   35.333591  -3.7368  -9.1006
68     Topic5    0.139540             php   35.690485  -3.7379  -9.0916
82     Topic5    0.140340           style   35.959470  -3.7397  -9.0859
92     Topic5    0.138005            user   35.912485  -3.7552  -9.1027
29     Topic5    0.139273         example   37.9431